### 1) Which linear regresssion training algorithm can you use if you have a training set with millions of features?

In this case (and supposing you dont have millions more observations), you should opt for Gradient Descent Methods. Closed form solutions are infeasible because they require you to have at least as many observations as features (m > n) in order to $X^{T}X$ to be invertible. SVD also should be avoided since it would take too long to (because of the many features) calculate the pseudinverse. Hence, you shoul use gradient descent methods. More speciffically, you should use either SGD or Mini Batch Gradient Descent for efficiency.

### 2) Supppose the features in your training set have very different scales. Which algorithms might suffer from this, and how? What can you do about it?

Any gradient descent training algorithim would suffer, taking much longer to converge. That is because these algorithms use the same learning rate for every feature, hence, features with smaller scales take proportionally bigger steps at each iteration. To solve you should just scale your features, like using StandardScaler.  

### 3) Can gradient descent get stuck in a local minimum when training a logistic regression model?

No, since the cost function of a logistic regression is stricly convex.

### 4) Do all gradient descent algorithms lead to the same model, provided you let them run long enough?

If you let then run long enough, and the learning rate is not high enough so they diverge, they will all converge to the minimum or, in the case of SGD and Mini-Batch GD, cycle through the minimum. SGD and Mini_batch will converge if you decrease the learning rate at each epoch.

### 5) Suppose you use batch gradient descent and you plot the validation error at every epoch. If you notice that the validation error consistently goes up, what is likely going on? How can you fix this?

It depends; if the training error is going down, then the model is probably overfitting to the data. There are a couple of things you can do. First, you could use some kind of regularization to penalyze some weights and consequently overfitting, or yo could try to get more observations. If you notice that the validation error started by decreasing in the beggining and then started increasing, you could also use an early stop procedure. On the other hand, if the training error is also going up, then the learning rate is probably too high and you should first decrease that.

### 6) Is it a good idea to stop mini-batch gradient descent immediately when the balidation error goes up?

No. Since you are not using the full features dataset, it could be that in this specific iteration, the error has gone up because the observations are a bit more different than the ones already trained in the model, and so the error goes up and you should not stop. But if after some iterations the error continues to increase, then you should stop.

### 7) Which gradient descent algorithim (among those we discussed) will reach the vicinity of the optimal solution the fastest? Wchich will actually converge? How can you make the others converge as well?

The SGD. Since, in each epoch, SGD works with just one row of observations (instance), it is able to run each epoch much faster than Batch GD for example, but not necessarily it will converge. Batch GD will converge as long as the learning rate is not too big. One can make SGD and Mini-Batch GD converge by decreasing the learning rate over time.

### 8) Suppose you are using polynomial regression. You plot the learning curves and you notice that there is a large gap between the training error and the validation error. What is happening? What are three ways to solve this?

You are probably overfitting. You could use some regularization technique, like the reidge regression to penalyze weights and decrease overfitting. Additionaly, you could try to get more instances so your model has more training observations. Finally, you could use some grid search (with cross validation) to find a better suiting polynomial degree.

### 9) Suppose you are using ridge regression and you notice that the training error and the validation error are almost equal and fairly high. Would you say that the model suffers from high bias or high variance? Should you increase the regularization hyperparameter $\alpha$ or reduce it?

This model if probably suffering of high bias. Since the training and validation error are similar, the model is not overfitting. But because both errors are high, the estimation is poor. Hence, you should decrease $\alpha$ and allow a bit more of variance in order to decrease your bias.

### 10) Why would you want to use:
    a. Ridge regression instead of plain linear regression (i.e., without any regularization)?
    b. Lasso instead of ridge regression?
    c. Elastic new instead of lasso regression.

a. Even though linear regression is not too prone to overfitting, it could happen if you have a very wide dataset (too many features for too few instances). In this case, or any other with overfitting problems, you could try to decrease this overfit with a regularization technique, such as Ridge regression.
b. Lasso can be used for feature selection. That is, since lasso sets some weights to zero, not all features will have an impact on the estimation. So if you think not all featreus are important to predict, you should use lasso.
c. Elastic new is a middle ground between lasso and ridge, depending o the r ratio (l1_ratio), it will give results more similar to laso or to ridge. Hence, if you want a model that has some of the benefits of both, without so much of their problems (like lasso`s bad behaviour with few instances), you should opt for elastic net.

### 11) Suppose you want to classify pictures as outdoor/indoor and daytime/nighttime. Should you implement two logistic regression classifiers or one softmax regression classifier?

In this case you should run two logistic regression, of for outdoor/indoor and one for daytime/nighttime. That is the case because softmax is a multiclass classifier, and not multioutput. Since we have two outputs (location and time of day), you cant use softmax to classify both outputs at the same time.

### 12) Implement batch gradient descent with early stopping for softmax regression without using Scikit-Learn, only Numpy. Use it on a classification task such as the iris dataset.

In [172]:
import numpy as np
class softmax_classification:
    def __init__(self, X, y, epochs = 5000, eta = 0.5):
        self.X = X
        #add columns of 1s for bias term
        self.X = np.c_[np.repeat(1, self.X.shape[0]), self.X]
        self.y = y
        self.K = len(np.unique(self.y)) #distinct label classes
        self.y_onehot = np.zeros((self.X.shape[0], self.K))
        self.y_onehot[np.arange(self.X.shape[0]), self.y] = 1 #one-hot encode y
        #Standardize X values
        self.meanX = np.mean(self.X[:,1:], axis = 0)
        self.stdX = np.std(self.X[:,1:], axis = 0)
        self.X[:,1:] = (self.X[:,1:] - self.meanX)/self.stdX
        #split into training and validation
        train_size = round(0.8 * self.X.shape[0])
        all_indices = np.array(range(self.X.shape[0]))
        train_indices = np.random.choice(all_indices, size = train_size, replace = False)
        valid_indices = np.setdiff1d(all_indices, train_indices)
        self.X_train = self.X[train_indices]
        self.X_valid = self.X[valid_indices]
        self.y_train = self.y[train_indices]
        self.y_valid = self.y[valid_indices]
        self.y_onehot_train = self.y_onehot[train_indices]
        self.y_onehot_valid = self.y_onehot[valid_indices]
        #additional parameters and matrixes
        self.epochs = epochs
        self.eta = eta
        self.m = self.X_train.shape[0] #number of observations (instances)
        self.n = self.X_train.shape[1] #number of parameters
        self.Theta = np.random.randn(self.n,self.K) #1 parameter row for each possible label
        self.scores = np.zeros(shape = (self.m, self.K)) #matrix to save scores for each instance
        self.probs = np.zeros(shape = (self.m, self.K)) #matrix to save probabilities for each instance
        self.gradients = np.zeros(shape = (self.m, self.K)) #initialize gradients

       
    #Calculate score for each possible label
    def softmax_score(self, X_use, Theta_use):
        scores = np.matmul(X_use, Theta_use)
        return(scores)

    #softmax function to calculate probabilities
    def softmax_function(self, scores_use):
        #exponentiate scores
        scores_exp = np.matrix(np.exp(scores_use))
        #divide by row sum
        probs = scores_exp/scores_exp.sum(axis = 1)
        return(probs)
    
    #calculate gradients
    def calc_gradient(self, probs_use):
        error = probs_use - self.y_onehot_train
        grad = (1/self.m)*np.matmul(self.X_train.T, error)
        return(grad)

    #update weights
    def update_weights(self):
        self.Theta = self.Theta - self.eta*self.gradients

    #Cross entropy-loss function
    def cross_entropy_loss(self):
        valid_scores = self.softmax_score(X_use = self.X_valid, Theta_use = self.Theta)
        probs_valid = self.softmax_function(scores_use = valid_scores)
        loss = np.multiply(self.y_onehot_valid, np.log(probs_valid)).sum(axis = 1).mean() * -1/(self.m)
        return(loss)

    #Predict method
    def predict(self, X_new):
        #add columns of 1s for bias term
        X_new_standard = (X_new - self.meanX)/self.stdX
        X_new_standard = np.c_[np.repeat(1, X_new_standard.shape[0]), X_new_standard]
        new_scores = self.softmax_score(X_use = X_new_standard, Theta_use = self.Theta)
        new_probs = self.softmax_function(scores_use = new_scores)
        return(new_probs.argmax(axis = 1))


    #fit
    def fit(self):
        min_loss = float('inf')
        for count in range(self.epochs):
            self.scores = self.softmax_score(X_use = self.X_train, Theta_use = self.Theta)
            self.probs = self.softmax_function(scores_use = self.scores)
            self.gradients = self.calc_gradient(probs_use = self.probs)
            self.update_weights()

            #calculate loss
            loss = self.cross_entropy_loss()
            if loss < min_loss:
                min_loss = loss
                best_Theta = self.Theta

            #Define best model as the one with minimum loss
            self.Theta = best_Theta


Now we should be able to open iris dataset and train the model

In [173]:
from sklearn.datasets import load_iris
iris = load_iris(as_frame = True)
X = iris.data[["petal length (cm)", "petal width (cm)"]].values
y = iris.target


#split into train and test
train_size = round(0.8 * X.shape[0])
all_indices = np.array(range(X.shape[0]))
train_indices = np.random.choice(all_indices, size = train_size, replace = False)
test_indices = np.setdiff1d(all_indices, train_indices)
X_train = X[train_indices]
X_test = X[test_indices]
y_train = np.array(y[train_indices])
y_test = np.array(y[test_indices])

#Fit the model
my_model = softmax_classification(X_train, y_train)
my_model.fit()

#Print accuracy
predictions = my_model.predict(X_test).T
accuracy = (predictions == y_test).mean()
print(f"Model test accuracy is: {round(accuracy,3)*100} %")



Model test accuracy is: 96.7 %
